<a href="https://colab.research.google.com/github/gagan3012/DetectingBias/blob/master/DetectingBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Feb 26 19:03:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!mkdir ~/.kaggle
!echo '{"username":"gagan2000","key":"73e5d9db85677b98fe016b25873727ec"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install -U -q kaggle

In [ ]:
!kaggle datasets download -d snapcrack/all-the-news -p /content

 94% 229M/244M [00:01<00:00, 155MB/s]
100% 244M/244M [00:01<00:00, 161MB/s]


In [ ]:
!unzip /content/all-the-news.zip -d /content/

Archive:  /content/all-the-news.zip
  inflating: /content/articles1.csv  
  inflating: /content/articles2.csv  
  inflating: /content/articles3.csv  


In [ ]:
!pip install tensorflow==1.15 #CPU
!pip install tensorflow-gpu==1.15  # GPU
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 512kB 47.5MB/s 
     |████████████████████████████████| 3.8MB 47.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=e42e39a4c31d34d67c046827416cea0a281718bf83fadb8b6ee64f50261922f6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [ ]:
gpt2.download_gpt2(model_name="124M")

gpt2.mount_gdrive()

Fetching checkpoint: 1.05Mit [00:00, 368Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.15Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 580Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:23, 21.2Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 179Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.87Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.42Mit/s]


Mounted at /content/drive


In [ ]:
d1 = pd.read_csv('articles1.csv', encoding = 'utf-8')
d2 = pd.read_csv('articles2.csv', encoding = 'utf-8')
d3 = pd.read_csv('articles3.csv', encoding = 'utf-8')
data = pd.concat([d1, d2, d3])
data.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [ ]:
data['publication'].value_counts()

Breitbart              23781
New York Post          17493
NPR                    11992
CNN                    11488
Washington Post        11114
Reuters                10710
Guardian                8681
New York Times          7803
Atlantic                7179
Business Insider        6757
National Review         6203
Talking Points Memo     5214
Vox                     4947
Buzzfeed News           4854
Fox News                4354
Name: publication, dtype: int64

In [ ]:
data['content'] = data['content'].fillna("")
data['title'] = data['title'].fillna("")

In [ ]:
final_right = data[data['publication'].str.contains("Fox News")]
final_right = final_right[['title','content']]
final_right.shape

(4354, 2)

In [ ]:
final_left = data[data['publication'].str.contains("CNN")]
final_left = final_left[['title','content']]
final_left.shape

(11488, 2)

In [ ]:
final_right['content'].to_csv("rightcontent.txt",header=None, index=None, sep='\t', mode='a')
final_left['content'].to_csv("leftcontent.txt",header=None, index=None, sep='\t', mode='a')

In [ ]:
sess = gpt2.start_tf_sess()
file_name_1 = "rightcontent.txt"

In [ ]:
#for right content
gpt2.finetune(sess,
              dataset=file_name_1,
              model_name='124M',
              steps=2000,
              restore_from='fresh',
              run_name='run_right_1',
              print_every=100,
              sample_every=200,
              save_every=500,
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:14<00:00, 14.76s/it]


dataset has 3017097 tokens
Training...
[100 | 236.44] loss=2.81 avg=2.81
[200 | 470.92] loss=2.73 avg=2.77
======== SAMPLE 1 ========
 congressional administration and was not authorized to publicly discuss his private emails. He then called up Clinton aide Huma Abedin, and asked her, “Did you email me, or did you not email me?” Clinton replied, “No, sir, I’m sorry.” The Justice Department has been trying to get Comey under the federal Espionage Act to give him immunity in exchange for testimony about his conversations with the   about the email system, even as several media organizations are pushing the theory of obstruction. “There’s no question that the FBI is investigating Mr. Comey for his alleged conduct in conducting a review of the Clinton Foundation and for his statements regarding his contacts with foreign officials, and for failing to disclose any contacts with foreign government officials, which is part of his request to Congress,” Rep. Dave Reichert, of Wisconsin, said. “T

In [ ]:
gpt2.generate(sess, run_name='run_right_1',
              length=100,
              temperature=.7,
              nsamples=10,
              batch_size=10,
              prefix="Allowing more immigrants to come into the United States will",
              truncate="<|endoftext|>",
              include_prefix=False
              )

Allowing more immigrants to come into the United States will lead to more job losses, lower wages, and even bankrupt our economy, all of which could have been avoided by allowing more immigrants to come into the country.” The bill was introduced in a party line vote on the House floor and then voted on the Senate floor for                            Y.) In reply to a question on immigration, House Speaker Paul Ryan,  . Y
Allowing more immigrants to come into the United States will safeguard our nation from the worst effects of immigration.” The                       Trump said he’d be the “greatest leader ever” of all, but added he’d have to “come back and fix our system. ” “We have to be able to take care of the country,” he said. “I want to work with Hillary
Allowing more immigrants to come into the United States will have huge economic and social consequences. It will create thousands of jobs, and bring in millions of new people. And it will also have the greatest impact on the econ

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run_right_1')

In [ ]:
ses = gpt2.start_tf_sess()
file_name_2 = "leftcontent.txt"

In [ ]:
#for left content
gpt2.finetune(ses,
              dataset=file_name_2,
              model_name='124M',
              steps=100,
              restore_from='fresh',
              run_name='run_left_1',
              print_every=10,
              sample_every=200,
              save_every=500,
              )

ValueError: ignored

In [ ]:
gpt2.generate(sess, run_name='run_left_1',
              length=100,
              temperature=.7,
              nsamples=10,
              batch_size=10,
              prefix="Allowing more immigrants to come into the United States will",
              truncate="<|endoftext|>",
              include_prefix=True
              )